## 作業目標: 使用Jieba進行各項的斷詞操作

這份作業我們會使用中文飯店評論資料集來作為斷詞練習。 [資料集:ChnSentiCorp_htl_all](https://github.com/SophonPlus/ChineseNlpCorpus)

### 讀入文本資料

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import pandas as pd

# hint: 可利用pandas讀取CSV
pd_corpus = pd.read_csv('/content/drive/MyDrive/NLP/ChnSentiCorp_htl_all.csv')

pd_corpus.head(5)

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"


In [2]:
#確認所有留言,正評價(label=1)與負評價數量(label=0)
pd_positive=pd_corpus[pd_corpus['label']==1]
pd_negative=pd_corpus[pd_corpus['label']==0]
print(f'Total: {len(pd_corpus)}, Positive: {len(pd_positive)}, Negative: {len(pd_negative)}')

Total: 7766, Positive: 5322, Negative: 2444


In [3]:
#移除缺失值
print(pd_corpus.info()) 
pd_corpus.dropna(subset=['review'],inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7766 entries, 0 to 7765
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   7766 non-null   int64 
 1   review  7765 non-null   object
dtypes: int64(1), object(1)
memory usage: 121.5+ KB
None


In [ ]:
pd_corpus[pd_corpus['review'].isnull()]

,label,review


### 建構結巴斷詞Function

建構可將讀入的Pandas DataFrame的文本資料，外加一欄名為`cut`的review欄位斷詞結果

In [4]:
import jieba

In [5]:
class JiebaCutingClass(object):
    '''Class to use jeiba to parse corpus from dataframe and cut the corpus
    
    Parameters
    -----------
    key_to_cut: str
        the dataframe key to parse the sentence for jieba cutting
    dic: str
        the dictionary to use for jieba, default is None (use default dictionary)
    userdict: str
        the user defined dictionary to use for jieba, default is None
    '''
    
    def __init__(self, key_to_cut:str, dic:str=None, userdict:str=None):
        
        if dic is not None:
            jieba.set_dictionary(dic)
        
        if userdict is not None:
            jieba.load_userdict(userdict)
        
        self.key_to_cut = key_to_cut
        
        #將paddle 功能開啟
        jieba.enable_paddle()
        
        
    @staticmethod
    def cut_single_sentence(sentence, use_paddle=False, use_full=False, use_search=False):
        
        if use_search:
            # hint:使用收尋引擎模式進行斷詞
            out = [word for word in jieba.cut_for_search(sentence)]
        else:
            # hint:非搜尋引擎模式斷詞，請注意有精確模式、全模式與paddle模式
            out = [word for word in jieba.cut(sentence,cut_all=use_full,use_paddle=use_paddle)]
        
        return out
            
    
    def cut_corpus(self, corpus: pd.DataFrame, mode: str) -> pd.DataFrame:
        '''Method to read and cut sentence from dataframe and append another column named cut
        
        Paremeters
        --------------
        corpus: pd.DataFrame
            Input corpus in dataframe
        mode: str
            Jieba mode to be used
        
        Return
        ----------------
        out: pd.Dataframe
            Output corpus in dataframe
        '''
        
        # checking valid mode
        if mode not in ['paddle', 'full', 'precise', 'search']:
            raise TypeError(f'only support `paddle`, `full`, `precise`, and `search` mode, but get {mode}')
            
        # cut the corpus based on mode
        # hint: 根據mode來呼叫相對應的斷詞method
        if mode == 'paddle':
            out = self._paddle_cut(corpus)
        elif mode == 'full':
            out = self._full_cut(corpus)
        elif mode == 'precise':
            out = self._precise_cut(corpus)
        elif mode == 'search':
            out = self._search_cut(corpus)

        return out
    
    def _paddle_cut(self, corpus):
        '''paddle mode
        '''
        #enable paddle
        #hint:先啟用paddle mode
        #jieba.enable_paddle()
        
        out = []
        # hint:將讀進的文本進行斷詞，並將結果append回out的陣列中
        for single_review in corpus[self.key_to_cut]:
            single_cut = [word for word in jieba.cut(single_review,use_paddle=True)]
            out.append(single_cut)
        corpus['cut'] = out
        
        return corpus
    
    def _full_cut(self, corpus):
        '''full mode
        '''
        
        out = []
        # hint:將讀進的文本進行斷詞，並將結果append回out的陣列中
        for single_review in corpus[self.key_to_cut]:
            single_cut = [word for word in jieba.cut(single_review,cut_all=True)]
            out.append(single_cut)
        
        corpus['cut'] = out
        
        return corpus
    
    def _precise_cut(self, corpus):
        '''precise mode
        '''
        
        out = []
        # hint:將讀進的文本進行斷詞，並將結果append回out的陣列中
        for single_review in corpus[self.key_to_cut]:
            single_cut = [word for word in jieba.cut(single_review)]
            out.append(single_cut)
        
        corpus['cut'] = out
        
        return corpus
    
    def _search_cut(self, corpus):
        '''search mode
        '''
        
        out = []
        # hint:將讀進的文本進行斷詞，並將結果append回out的陣列中
        for single_review in corpus[self.key_to_cut]:
            single_cut = [word for word in jieba.cut_for_search(single_review)]
            out.append(single_cut)
        
        corpus['cut'] = out
        
        return corpus

### 使用建構好的斷詞物件對文本進行斷詞

In [9]:
### 請使用精確模式與預設字典對文本進行斷詞

# hint:請先實例化JiebaCutingClass，再進行斷詞
Jiebacut = JiebaCutingClass('review')
###<your code>### #為了避免處理時間過久, 這裡我們只使用前50個進行斷詞
pd_cut = Jiebacut.cut_corpus(pd_corpus[:50],'paddle')
pd_cut.head()

Paddle enabled successfully......
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,label,review,cut
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较...","[距离, 川沙公路, 较, 近, ,, 但是, 公交, 指示, 不对, ,, 如果, 是, ..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!,"[商务, 大床, 房, ，, 房间, 很大, ，床, 有, 2M, 宽, ，, 整体, 感觉..."
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。,"[早餐, 太, 差，, 无, 论, 去, 多少, 人, ，, 那边, 也, 不, 加, 食品..."
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...,"[宾馆, 在, 小街道, 上, ，不大好找, ，, 但, 还好, 北京, 热心, 同胞, 很..."
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风","[CBD中心, ,, 周围, 没, 什么, 店铺, ,, 说, 5, 星, 有点, 勉强, ..."


### 載入繁中字典為指定的字詞進行斷詞

In [ ]:
test_string = '我愛cupoy自然語言處理馬拉松課程'
jieba_cut = JiebaCutingClass(key_to_cut='', dic='/content/drive/MyDrive/NLP/dict.txt.big')


out_string = cut_single_sentence(sentence, use_paddle=True)
print(f'Paddle模式: {[string for string in out_string]}')

out_string = cut_single_sentence(sentence, use_full=True)
print(f'全模式: {[string for string in out_string]}')

out_string = cut_single_sentence(sentence, use_search=True)
print(f'搜尋模式: {[string for string in out_string]}')

out_string = cut_single_sentence(sentence)
print(f'精確模式: {[string for string in out_string]}')

Paddle enabled successfully......
Building prefix dict from /Users/admin/Documents/cupoy/Ch06-Jieba/dict.txt.big ...
Loading model from cache /var/folders/9f/mkwhwg7d4vz7rp0429_kg43c0000gn/T/jieba.u7693c7663763ede1f644a996682c676f.cache


Paddle模式: ['我', '愛', 'cupoy', '自然', '語言', '處理', '馬拉松', '課程']


Loading model cost 1.685 seconds.
Prefix dict has been built successfully.


全模式: ['我', '愛', 'cupoy', '自然', '自然語言', '語言', '處理', '馬拉', '馬拉松', '課程']
搜尋模式: ['我', '愛', 'cupoy', '自然', '語言', '自然語言', '處理', '馬拉', '馬拉松', '課程']
精確模式: ['我', '愛', 'cupoy', '自然語言', '處理', '馬拉松', '課程']


In [44]:
test_string = '我愛cupoy自然語言處理馬拉松課程'
jieba_cut = JiebaCutingClass(key_to_cut='', dic='/content/drive/MyDrive/NLP/dict.txt.big')


out_string = Jiebacut.cut_single_sentence(test_string, use_paddle=True)
print(f'Paddle模式: {[string for string in out_string]}')

out_string = Jiebacut.cut_single_sentence(test_string, use_full=True)
print(f'全模式: {[string for string in out_string]}')

out_string = Jiebacut.cut_single_sentence(test_string, use_search=True)
print(f'搜尋模式: {[string for string in out_string]}')

out_string = Jiebacut.cut_single_sentence(test_string)
print(f'精確模式: {[string for string in out_string]}')

Paddle enabled successfully......
Building prefix dict from /content/drive/MyDrive/NLP/dict.txt.big ...


Paddle模式: ['我', '愛', 'cupoy', '自然', '語言', '處理', '馬拉松', '課程']


Dumping model to file cache /tmp/jieba.u67cbe97c857ec9809d2a6fb43fe09b0f.cache
Loading model cost 2.563 seconds.
Prefix dict has been built successfully.


全模式: ['我', '愛', 'cupoy', '自然', '自然語言', '語言', '處理', '馬拉', '馬拉松', '課程']
搜尋模式: ['我', '愛', 'cupoy', '自然', '語言', '自然語言', '處理', '馬拉', '馬拉松', '課程']
精確模式: ['我', '愛', 'cupoy', '自然語言', '處理', '馬拉松', '課程']
